# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [39]:
# import libraries
from sqlalchemy import create_engine

import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /home/jens/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jens/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jens/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jens/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_database.db')
df = pd.read_sql('messages', engine)
df.head()
X = df['message']
Y = df.drop(labels=['genre', 'id', 'message', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) 
    clean_tokens = []
    
 
    filtered_tokens = [w for w in tokens if not w in stop_words]  
    
    for tok in filtered_tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

for i in range(10):
    print(tokenize(df['message'][i]))

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
['is', 'hurricane']
['looking', 'someone', 'name']
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.']
['say', ':', 'west', 'side', 'haiti', ',', 'rest', 'country', 'today', 'tonight']
['information', 'national', 'palace-']
['storm', 'sacred', 'heart', 'jesus']
['please', ',', 'need', 'tent', 'water', '.', 'we', 'silo', ',', 'thank', '!']
['i', 'would', 'like', 'receive', 'message', ',', 'thank']
['i', 'croix-des-bouquets', '.', 'we', 'health', 'issue', '.', 'they', '(', 'worker', ')', 'santo', '15', '.', '(', 'area', 'croix-des-bouquets', ')']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])



In [36]:
print(pipeline)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.1,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='entropy',
                                                               

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
parameters = {
    'clf__estimator__n_estimators': 100,
    'clf__estimator__criterion': 'entropy',
    'clf__estimator__max_features': ('auto'),#, 'log2'),
    'clf__estimator__min_samples_split': 2,
    'vect__ngram_range':  (1, 2),
    'tfidf__use_idf': False,
    'vect__max_df': 0.1
}

pipeline.set_params(**parameters)

pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [37]:
print(pipeline.score(X_test, Y_test)) #score old 0.23605

0.2638655462184874


In [32]:
Y_test_array = Y_test.to_numpy().T
Y_test_array.shape
Y_pred.T.shape


(35, 6545)

In [33]:
Y_pred.T[2].shape

(6545,)

In [34]:
for i, column in enumerate(Y_pred.T):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [44]:
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    print('roc auc score', roc_auc_score(Y_test_array[column], Y_pred.T[column]))
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

           0       0.73      0.38      0.50      1529
           1       0.83      0.96      0.89      5016

    accuracy                           0.82      6545
   macro avg       0.78      0.67      0.70      6545
weighted avg       0.81      0.82      0.80      6545

roc auc score 0.6679359861393183
[[ 580  949]
 [ 218 4798]]
request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5436
           1       0.88      0.45      0.59      1109

    accuracy                           0.90      6545
   macro avg       0.89      0.72      0.77      6545
weighted avg       0.90      0.90      0.88      6545

roc auc score 0.7180051037368351
[[5370   66]
 [ 612  497]]
offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        19

    accuracy                           1

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


roc auc score 0.5103554647517268
[[6306    3]
 [ 231    5]]
death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6243
           1       0.89      0.11      0.19       302

    accuracy                           0.96      6545
   macro avg       0.92      0.55      0.58      6545
weighted avg       0.96      0.96      0.94      6545

roc auc score 0.552659773648473
[[6239    4]
 [ 270   32]]
other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5654
           1       0.69      0.03      0.05       891

    accuracy                           0.87      6545
   macro avg       0.78      0.51      0.49      6545
weighted avg       0.84      0.87      0.81      6545

roc auc score 0.513056418049933
[[5643   11]
 [ 866   25]]
infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6086
           1       0

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
from pprint import pprint
pprint(pipeline.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators='warn',
                                                       n_jobs=None,
         

In [25]:
parameters = {
    #'clf__estimator__n_estimators': (10, 100),
    'clf__estimator__criterion': ('gini', 'entropy'),
    'clf__estimator__max_features': ('auto', 'sqrt'),#, 'log2'),
    'clf__estimator__min_samples_split': (2, 3),#,
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': (True, False),
    'vect__max_df': (0.1, 0.5, 1.0)
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=6, verbose=5)
grid_fit = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed: 10.6min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed: 25.0min
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed: 48.4min
[Parallel(n_jobs=6)]: Done 288 out of 288 | elapsed: 50.4min finished
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will ch

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
grid_fit.best_score_

0.24419315403422984

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.